## Step2 with ErgoSCF

In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn
from libra_py import ERGO_methods
from libra_py import units
from libra_py.workflows.nbra import step2_ergoscf


/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator

Define the function to run the ErgosCF calculations for each geometry. This function essentially defines the core of the input file to the ErgoSCF code. Only the coordinates and the name of the executable may be changed

In [9]:
def run(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_files_F = 1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Also define how to run the calculations for the "weird" geometry, when two snapshots are overlapped and form a "super-molecule". In this case, we don't really need to print out the Fock matrices

In [ ]:
def run1(EXE, COORDS):
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline
%sEOF
basis = "STO-3G"
use_simple_starting_guess=1
scf.create_mtx_file_S = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Here, lets define another one (still under development) - to also compute the nuclear gradients on the converged charge density. This function may be important to run an on-the-fly dynamics

In [ ]:
def run2(EXE, COORDS):
    """
    This function also requests computation of the nuclear gradients
    """
    
    inp = """#!bin/sh
%s << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline
%sEOF
basis = "STO-3G"
scf.compute_gradient_fixeddens = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT
""" % (EXE, COORDS)
    return inp

Ok, so now we can run the step2 calculations! Just create the "res" dir, since this is where the results will be printed out

In [5]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz",
          "isnap":0, "fsnap":5, "dt": 1.0 * units.fs2au
         }

os.system("mkdir res")
step2_ergoscf.run_step2(params, run, run1)

## Other examples

If we jneed to run an SCF calculations for a singe geometry

In [8]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz"}
i = 0

E, MO = step2_ergoscf.do_step(i, params, run2)

And we can print out the resulting matrices into a file

In [ ]:
E.show_matrix("E0.txt")
MO.show_matrix("MO.txt")

Here is another example - just to submit the ErgoSCF calculation for a given geometry and run type

In [10]:
EXE = "/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo"
R = ERGO_methods.xyz_traj2gen_sp("md1.xyz", 0)
command = run2(EXE, R)
print command
os.system("%s" % (command))

#!bin/sh
/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo << EOINPUT > /dev/null
spin_polarization = 0
molecule_inline
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
EOF
ghost_inline
 Li      0.00000000       0.00000000       0.00000000
 H      0.00000000       0.00000000       1.33200000
EOF
basis = "none"
ghost_basis = "STO-3G"
scf.compute_gradient_fixeddens = 1
XC.sparse_mode = 1
run "LDA"
EOINPUT



0

We can read the produced matrices - in this case the Fock matrices. And we can specify which section of the matrix we want to read

In [ ]:
F = ERGO_methods.get_mtx_matrices("F_matrix_11.mtx")
F.show_matrix("F.txt")

F3 = ERGO_methods.get_mtx_matrices("F_matrix_11.mtx", [0,1,2], [0,1,2])
F3.show_matrix("Fa.txt")

Among a series of the Fock matrices, find the last one

In [ ]:
print ERGO_methods.find_last_file("F_matrix_", ".mtx")

Get a geometry for a "super-molecule" composed of two snapshots

In [ ]:
i = 0
R = ERGO_methods.xyz_traj2gen_ovlp("md1.xyz", i, i+1)
print R

Here is an example to run the calculation of the overlap matrix between AOs in the two adjacent geometries

In [ ]:
params = {"EXE":"/mnt/c/cygwin/home/Alexey-user/Soft/ergo-3.7/source/ergo", "md_file":"md1.xyz"}
i = 0

S = step2_ergoscf.do_ovlp(i, params, run1)
S.show_matrix("S.txt")